In [9]:
import numpy, re

import iris
from iris.util import rolling_window
from scipy import stats

In [10]:
infile = '/g/data/r87/dbi599/data_argo/ohc-metrics-globe60equiv_Omon_ArgoRoemmich_2004-2015.nc'

In [11]:
def calc_diff_trends(sthext_cube, notsthext_cube, window=120):
    """Calculate trends in difference between southern extratropics and rest of globe.

    A window of 120 matches the length of the Argo record 
      (i.e. 10 years of annually smoothed monthly data)

    """

    diff = sthext_cube - notsthext_cube
    diff_windows = rolling_window(diff.data, window=window, axis=0)    
    x_axis_windows = rolling_window(diff.coord('time').points, window=window, axis=0)

    ntimes = diff_windows.shape[0]
    trends = numpy.zeros(ntimes)
    for i in range(0, ntimes):
        x = x_axis_windows[i, :]
        y = diff_windows[i, :]
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        trends[i] = slope

    # convert units from J/month to J/s so can be expressed as Watts (1 J = W.s)
    assert 'days' in str(diff.coord('time').units)
    hours_in_day = 24
    minutes_in_hour = 60
    seconds_in_minute = 60

    trends = trends / (hours_in_day * minutes_in_hour * seconds_in_minute)

    return trends

In [12]:
def get_time_constraint(time_list):
    """Get the time constraint used for reading an iris cube."""
    
    start_date, end_date = time_list

    date_pattern = '([0-9]{4})-([0-9]{1,2})-([0-9]{1,2})'
    assert re.search(date_pattern, start_date)
    assert re.search(date_pattern, end_date)

    if (start_date == end_date):
        year, month, day = start_date.split('-')    
        time_constraint = iris.Constraint(time=iris.time.PartialDateTime(year=int(year), month=int(month), day=int(day)))
    else:  
        start_year, start_month, start_day = start_date.split('-') 
        end_year, end_month, end_day = end_date.split('-')
        time_constraint = iris.Constraint(time=lambda t: iris.time.PartialDateTime(year=int(start_year), month=int(start_month), day=int(start_day)) <= t.point <= iris.time.PartialDateTime(year=int(end_year), month=int(end_month), day=int(end_day)))

    return time_constraint

In [14]:
time_constraint = get_time_constraint(['2006-01-01', '2015-12-31'])

with iris.FUTURE.context(cell_datetime_objects=True):
    cube_sthext = iris.load_cube(infile, 'ocean heat content southern extratropics60' & time_constraint)
    cube_notsthext = iris.load_cube(infile, 'ocean heat content northern extratropics60' & time_constraint)

In [15]:
print cube_sthext

ocean heat content southern extratropics60 / (10^22 J) (time: 120)
     Dimension coordinates:
          time                                              x
     Attributes:
          Conventions: CF-1.5
          comment: ARGO TEMPERATURE ANOMALY defined by Jan 2004 - Dec 2014 (11.0 year) RG...
          depth_bounds: ohc integrated over input data surface (2.500000) down to 2000.000000
          history: Thu Jun 09 10:26:12 2016: /g/data/r87/dbi599/miniconda2/envs/default/bin/python...
          invalid_standard_name: ocean_heat_content_southern_extratropics60


In [16]:
cube_sthext = cube_sthext.rolling_window('time', iris.analysis.MEAN, 12)
cube_notsthext = cube_notsthext.rolling_window('time', iris.analysis.MEAN, 12)

In [17]:
print cube_sthext

ocean heat content southern extratropics60 / (10^22 J) (time: 109)
     Dimension coordinates:
          time                                              x
     Attributes:
          Conventions: CF-1.5
          comment: ARGO TEMPERATURE ANOMALY defined by Jan 2004 - Dec 2014 (11.0 year) RG...
          depth_bounds: ohc integrated over input data surface (2.500000) down to 2000.000000
          history: Thu Jun 09 10:26:12 2016: /g/data/r87/dbi599/miniconda2/envs/default/bin/python...
          invalid_standard_name: ocean_heat_content_southern_extratropics60
     Cell methods:
          mean: time


In [18]:
diff_trend = calc_diff_trends(cube_sthext, cube_notsthext, window=109)

In [20]:
print diff_trend * 1e8

[ 4.13817064]
